In [1]:
!pip install pandas kagglehub ray[air] torch torchvision ray[tune] Pillow torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 927.3/927.3 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.9/274.9 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 76.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 87.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.6/4

In [2]:
import os
import ray
import ray.data as rd
import numpy as np
from ray import train, tune
from ray.air.config import ScalingConfig
from ray.train.torch import TorchTrainer
import torchvision.transforms as transforms
import torch
import torch.nn as nn
import torch.optim as optim
from PIL import Image
import kagglehub
from torch.utils.data import DataLoader
from typing import Dict, Any

ray.init(ignore_reinit_error=True)

2025-03-01 12:46:20,071	INFO worker.py:1832 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


Python version:,3.11.11
Ray version:,2.43.0
Dashboard:,http://127.0.0.1:8265


In [3]:
KAGGLE_DATASET = "ifigotin/imagenetmini-1000"
DATASET_DIR = os.path.expanduser("~/.cache/kagglehub/datasets/ifigotin/imagenetmini-1000/versions/1")

if not os.path.exists(DATASET_DIR):
  DATASET_DIR = kagglehub.dataset_download(KAGGLE_DATASET)

100%|██████████| 3.92G/3.92G [00:23<00:00, 181MB/s]

Extracting files...


In [4]:
# image transformations
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def load_image(row):
  image_path = row["item"]
  class_label = os.path.basename(os.path.dirname(image_path))
  img = Image.open(image_path).convert("RGB")
  img_array = np.copy(np.asarray(img))
  return {"image": Image.fromarray(img_array), "class_label": class_label}

def encode_labels(ds, label_to_idx):
    return ds.map(lambda x: {
        "image": x["image"],
        "class_label": label_to_idx[x["class_label"]]
    })

def preprocess_dataset(ds):
    return ds.map(lambda x: {"image": transform(x["image"]), "class_label": x["class_label"]})

def split_dataset(ds):
    return ds.train_test_split(test_size=0.2, shuffle=True)

def create_dataset(path):
    image_formats = ('.png', '.jpg', '.jpeg')
    return rd.from_items(
        [os.path.join(root, f)
         for root, _, files in os.walk(path)
         for f in files if f.lower().endswith(image_formats)],
        override_num_blocks=16
    ).map(load_image).filter(lambda x: x["image"] is not None and x["class_label"] is not None)

In [51]:
ds = create_dataset(DATASET_DIR)

In [5]:
ds = create_dataset(DATASET_DIR)
class_labels = sorted(ds.unique("class_label"))
label_to_num = {label: i for i, label in enumerate(class_labels)}
ds = encode_labels(ds, label_to_num)
ds = preprocess_dataset(ds)
ds = ds.random_sample(fraction=0.1)
train_ds, test_ds = split_dataset(ds)



/usr/local/lib/python3.11/dist-packages/ray/data/dataset.py:1330: UserWarning: Use 'expr' instead of 'fn' when possible for performant filters.
  warnings.warn(
2025-03-01 12:47:10,560	INFO dataset.py:2787 -- Tip: Use `take_batch()` instead of `take() / show()` to return records in pandas or numpy batch format.
2025-03-01 12:47:10,566	INFO streaming_executor.py:108 -- Starting execution of Dataset. Full logs are in /tmp/ray/session_2025-03-01_12-46-15_217417_3251/logs/ray-data
2025-03-01 12:47:10,567	INFO streaming_executor.py:109 -- Execution plan of Dataset: InputDataBuffer[Input] -> TaskPoolMapOperator[Map(load_image)->Filter(<lambda>)] -> AllToAllOperator[Aggregate] -> LimitOperator[limit=1]


Running 0: 0.00 row [00:00, ? row/s]

- Map(load_image)->Filter(<lambda>) 1: 0.00 row [00:00, ? row/s]

- Aggregate 2: 0.00 row [00:00, ? row/s]

Sort Sample 3:   0%|          | 0.00/1.00 [00:00<?, ? row/s]

...

2025-03-01 12:48:44,576	INFO streaming_executor.py:108 -- Starting execution of Dataset. Full logs are in /tmp/ray/session_2025-03-01_12-46-15_217417_3251/logs/ray-data
2025-03-01 12:48:44,577	INFO streaming_executor.py:109 -- Execution plan of Dataset: InputDataBuffer[Input] -> AllToAllOperator[Map(load_image)->Filter(<lambda>)->Map(<lambda>)->Map(<lambda>)->MapBatches(random_sample)->RandomShuffle]


Running 0: 0.00 row [00:00, ? row/s]

- Map(load_image)->Filter(<lambda>)->Map(<lambda>)->Map(<lambda>)->...->RandomShuffle 1: 0.00 row [00:00, ? ro…

Shuffle Map 2:   0%|          | 0.00/1.00 [00:00<?, ? row/s]

Shuffle Reduce 3:   0%|          | 0.00/1.00 [00:00<?, ? row/s]

In [9]:
print(f"Training samples: {train_ds.count()}, Test samples: {test_ds.count()}")


Training samples: 3061, Test samples: 797


In [7]:
train_ds.show(1)

2025-03-01 12:49:37,445	INFO streaming_executor.py:108 -- Starting execution of Dataset. Full logs are in /tmp/ray/session_2025-03-01_12-46-15_217417_3251/logs/ray-data
2025-03-01 12:49:37,446	INFO streaming_executor.py:109 -- Execution plan of Dataset: InputDataBuffer[Input] -> LimitOperator[limit=1]


Running 0: 0.00 row [00:00, ? row/s]

- limit=1 1: 0.00 row [00:00, ? row/s]

{'image': array([[[-0.95342064, -1.004795  , -0.9876702 , ..., -0.25130582,
         -0.11430778, -0.2170563 ],
        [-0.8677969 , -0.9020464 , -0.95342064, ..., -0.2170563 ,
         -0.23418105, -0.11430778],
        [-0.81642264, -0.88492167, -0.8506721 , ..., -0.26843056,
         -0.25130582, -0.18280679],
        ...,
        [-0.9020464 , -0.91917115, -0.91917115, ...,  0.14256364,
         -0.06293353, -0.13143253],
        [-0.78217316, -0.5424266 , -0.9362959 , ...,  0.2281874 ,
          0.12543888,  0.07406463],
        [-0.8677969 , -0.43967807, -0.91917115, ...,  0.33093593,
          0.33093593,  0.2966864 ]],

       [[ 0.3452382 ,  0.3102242 ,  0.29271722, ...,  0.8529412 ,
          0.83543426,  0.81792724],
        [ 0.3627452 ,  0.2752102 ,  0.3102242 , ...,  0.81792724,
          0.83543426,  0.76540625],
        [ 0.3452382 ,  0.3102242 ,  0.3102242 , ...,  0.8004202 ,
          0.8004202 ,  0.76540625],
        ...,
        [-0.37254897, -0.16246496, -0.092436

In [10]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes: int):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.classifier = nn.Sequential(
            nn.Linear(32*56*56, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        return self.classifier(x)

In [12]:
def train_epoch(model, loader, device,optimizer, criterion):
    model.train()
    total_loss = 0.0
    total_samples = 0
    for batch in loader:
        optimizer.zero_grad()
        # The loader returns a tuple: (features, labels)
        images, labels = batch
        # Remove the extra dimension if images are [batch, 1, 3, 224, 224]
        if images.dim() == 5 and images.shape[1] == 1:  # Check for the extra dimension
            images = images.squeeze(1)                  # Remove the extra dimension
        images = images.to(device)
        # Flatten labels to ensure they are 1D
        labels = labels.to(device).view(-1)
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * images.size(0)
        total_samples += images.size(0)
    return total_loss / total_samples

def train_func(config):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    train_shard = train.get_dataset_shard("train")

    train_loader = train_shard.to_torch(
        label_column="class_label",
        feature_columns=["image"],
        batch_size=config["batch_size"]
    )

    model = SimpleCNN(config["num_classes"])
    model.to(device)
    optimizer = optim.AdamW(model.parameters(), lr=config["lr"], weight_decay=1e-4)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(config["num_epochs"]):
        avg_loss = train_epoch(model, train_loader, device, optimizer, criterion)
        train.report({"loss": avg_loss, "epoch": epoch})

In [13]:
trainer = TorchTrainer(
    train_func,
    train_loop_config={
        "lr": 3e-4,
        "batch_size": 64,
        "num_epochs": 60,
        "num_classes": len(class_labels),
    },
    scaling_config=ScalingConfig(
        num_workers=32,

    ),
    datasets={"train": train_ds}
)

# Execute training
result = trainer.fit()
print(f"Final training results: {result.metrics}")


View detailed results here: /root/ray_results/TorchTrainer_2025-03-01_12-50-02
To visualize your results with TensorBoard, run: `tensorboard --logdir /tmp/ray/session_2025-03-01_12-46-15_217417_3251/artifacts/2025-03-01_12-50-02/TorchTrainer_2025-03-01_12-50-02/driver_artifacts`

Training started with configuration:
+----------------------------------------+
| Training config                        |
+----------------------------------------+
| train_loop_config/batch_size        64 |
| train_loop_config/lr            0.0003 |
| train_loop_config/num_classes     1000 |
| train_loop_config/num_epochs        60 |
+----------------------------------------+


(RayTrainWorker pid=18769) Setting up process group for: env:// [rank=0, world_size=32]
(TorchTrainer pid=18661) Started distributed worker processes: 
(TorchTrainer pid=18661) - (node_id=920afc47c19e633196951971553e96432b3c23501264371b697b6d64, ip=172.28.0.12, pid=18769) world_rank=0, local_rank=0, node_rank=0
(TorchTrainer pid=18661) - (node_id=920afc47c19e633196951971553e96432b3c23501264371b697b6d64, ip=172.28.0.12, pid=18771) world_rank=1, local_rank=1, node_rank=0
(TorchTrainer pid=18661) - (node_id=920afc47c19e633196951971553e96432b3c23501264371b697b6d64, ip=172.28.0.12, pid=18772) world_rank=2, local_rank=2, node_rank=0
(TorchTrainer pid=18661) - (node_id=920afc47c19e633196951971553e96432b3c23501264371b697b6d64, ip=172.28.0.12, pid=18768) world_rank=3, local_rank=3, node_rank=0
(TorchTrainer pid=18661) - (node_id=920afc47c19e633196951971553e96432b3c23501264371b697b6d64, ip=172.28.0.12, pid=18770) world_rank=4, local_rank=4, node_rank=0
(TorchTrainer pid=18661) - (node_id=920afc4

(pid=21588) Running 0: 0.00 row [00:00, ? row/s]

(pid=21588) - split(32, equal=True) 1: 0.00 row [00:00, ? row/s]

(RayTrainWorker pid=18771) /usr/local/lib/python3.11/dist-packages/ray/air/_internal/torch_utils.py:91: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:206.)
(RayTrainWorker pid=18771)   return torch.as_tensor(vals, dtype=dtype)


...

(pid=21588) Running 0: 0.00 row [00:00, ? row/s]

(pid=21588) - split(32, equal=True) 1: 0.00 row [00:00, ? row/s]


Training finished iteration 60 at 2025-03-01 12:56:39. Total running time: 6min 36s
+-------------------------------+
| Training result               |
+-------------------------------+
| checkpoint_dir_name           |
| time_this_iter_s      6.43204 |
| time_total_s          391.225 |
| training_iteration         60 |
| epoch                      59 |
| loss                  0.15327 |
+-------------------------------+


2025-03-01 12:56:40,948	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/root/ray_results/TorchTrainer_2025-03-01_12-50-02' in 0.0045s.



Training completed after 60 iterations at 2025-03-01 12:56:40. Total running time: 6min 37s

Final training results: {'loss': 0.153272419226797, 'epoch': 59, 'timestamp': 1740833798, 'checkpoint_dir_name': None, 'done': True, 'training_iteration': 60, 'trial_id': 'b10f7_00000', 'date': '2025-03-01_12-56-39', 'time_this_iter_s': 6.432043790817261, 'time_total_s': 391.2253601551056, 'pid': 18661, 'hostname': 'b21d84aef14d', 'node_ip': '172.28.0.12', 'config': {'train_loop_config': {'lr': 0.0003, 'batch_size': 64, 'num_epochs': 60, 'num_classes': 1000}}, 'time_since_restore': 391.2253601551056, 'iterations_since_restore': 60, 'experiment_tag': '0'}
